# Функция потерь (loss function) и функционал качества

### Напомним наши обозначения


Объект х из множества Х\
Для примера возьмем признаковое пространство (пол, рост)\
Дальше будем отождествлять объект х с его векторным представлением в виде  х = ($x_1$, $x_2$)\
Значит всего у нас 2 признака (2 компоненты), где $x_1$ это пол, $x_2$ это рост.


Сгенерируем вспомогательный датасет с которым и будем работать в этом воркшопе.

In [104]:
import numpy as np
import pandas as pd

np.random.seed(0)
#С библиотекой NumPy поближе мы познакомимся 
#в следующем уроке, здесь она нам понадобится 
#исключительно для того, чтобы сгенерировать случайные данные
#Признаками мы пока пользоваться не будем поэтому сгенерируем только пол и рост
df = pd.DataFrame({'пол': np.random.randint(2, size=1000), 
                   'рост':np.random.randint(low=100,high=250, size=1000)})

In [105]:
#посмотрим, что у нас получилось
df.head(10)

,пол,рост
0,0,139
1,1,223
2,1,128
3,0,246
4,1,145
5,1,154
6,1,100
7,1,238
8,1,234
9,1,199


Получился датасет, где мужской пол (1) и женский пол (0).
Также мы видим, что есть признак рост.

Эти данные размечены - нам известны значения признаков для каждого ряда.
Давайте тогда попробуем построить наивный алгоритм, который предсказывал бы для этого датасета рост.

Вспомним, что такое алгоритм

$a: X -> Y$ алгоритм, которым мы хотим приблизить целевую функцию\
А целевая функция это $y'(x):X_m -> Y$.
Ее значения известны только на тренировочном множестве объектов в нашем случае их 1000, значит здесь $m=1000$.
То есть здесь ее значения известны для каждого ряда датасета.

Попробуем решить сначала задачу предсказания роста человека.

#### Задача 1:
Для построенного датасета:\
    а) сделать предсказания роста для каждого наблюдения в нем\
    б) посчитать какое значение функционала качества дает наш алгоритм\

Алгоритм может быть как сложной функцией, так и просто константой (const), то есть числом.
Выберем алгоритм $а(х)$, который будет всегда возвращать число равное первому занчению роста в датафрейме.

In [106]:
df.iloc[0]['рост']

139

Это и будет наш алгоритм.
$a(x) = 139$

Оценим его качество и как он справился с поставленной задачей.
Для этого мы должны вспомнить как мерить ошибку на объектах.


$ L(x,a) = (a(x) - y'(x))^2 $ - для задач регрессии 


Значит мы можем написать вспомогательную функцию.

In [107]:
def loss_regression(y_predicted, y_true):
    return (y_predicted - y_true)**2

Она возвращает квадрат ошибки нашего алгоритма.
Давайте применим ее ко всем наблюдениям датафрейма.
А предсказание будем использовать из нашего алгоритма, тогда.

In [108]:
df['предсказанный_рост'] = df.iloc[0]['рост']

In [110]:
df.head()

,пол,рост,предсказанный_рост
0,0,139,139
1,1,223,139
2,1,128,139
3,0,246,139
4,1,145,139


Мы сделали наши первые предсказания, теперь посчитаем ошибку с помощью написанной функции.

In [111]:
df['функция_потерь_для_роста'] = df[['рост', 'предсказанный_рост']].apply(lambda x: loss_regression(x[1], x[0]), axis=1)

In [112]:
df.head()

,пол,рост,предсказанный_рост,функция_потерь_для_роста
0,0,139,139,0
1,1,223,139,7056
2,1,128,139,121
3,0,246,139,11449
4,1,145,139,36


Отлично!\
Теперь у нас есть ошибки для каждого объекта из датафрейма.\
Давайте теперь оценим насколько сильно мы ошибаемся в совокупности на всем датафрейме.\
Здесь нам пригодитя функционал качества или мы его также будем называть эмпирическим риском.

Вспомним как он выглядит:\
Эмпирический риск:\
$Q(a,X^{m}) = \frac{1}{m}\sum{L(x_{i}, a)}$,\
где m это количество объектов в обучающей выборке

In [113]:
def empirical_risk(loss):
    return sum(loss)/len(loss)

Все значения функции потерь у нас есть, теперь осталась просуммировать и поделить на количесвто объектов. Так и поступим.

In [130]:
df['функция_потерь_для_роста'].values.shape

(1000,)

In [115]:
empirical_risk(df['функция_потерь_для_роста'].values)

3279.074

Наша функция посчтитала значение эмпирического риска, видим что как-то уж очень далеко от нуля наше значение.\
Попробуем улучшить результат - давайте попробуем предсказывать средним ростом по датасету.

Посчитаем среднее.

In [135]:
#здесь нужно вставить код для подсчета среднего роста по датафрейму
df['предсказанный_рост'] = df['рост'].mean()

Посчитаем функцию потерь для каждого объекта.

In [136]:
#здесь нужно вставить код для подсчета функции потерь
df['функция_потерь_для_роста'] = df[['рост', 'предсказанный_рост']].apply(lambda x: loss_regression(x[1], x[0]), axis=1)

Теперь посчитаем эмпирический риск.

In [138]:
#Здесь нужно вставить код для подсчета функционала качества (эмпирического риска) - это и будет ответом
int(empirical_risk(df['функция_потерь_для_роста'].values))

1862

#### Задача 2:
Для построенного датасета:\
    а) сделать предсказания пола для каждого наблюдения в нем\
    б) посчитать какое значение функционала качества дает наш алгоритм





Выберем алгоритм $а(х)$, который будет всегда возвращать пол равный первому наблюдению пола в датафрейме.

In [119]:
df.iloc[0]['пол']

0

Это и будет значение нашего предсказания.
Мы видим, что это число типа float, но это никак не вляет на результат наших вычислений, поэтому оставим как есть.

Значение целевой переменной может принимать два вида значений это 0 или 1, значит перед нами задача бинарной классификации, то есть классификации на два класса.

Вспомним, какого вида тогда нужно выбрать функцию потерь.

$ L(x,a) = [a(x) != y'(x)] $  - для задач классификации

Отлично! Теперь можно писать вспомогательную функцию.

In [120]:
def loss_classify(y_predicted, y_true):
    return y_predicted != y_true

In [133]:
int(loss_classify(0, 1))

1

In [122]:
df['предсказанный_пол'] = df.iloc[0]['пол']

In [123]:
df.head()

,пол,рост,предсказанный_рост,функция_потерь_для_роста,предсказанный_пол
0,0,139,139,0,0
1,1,223,139,7056,0
2,1,128,139,121,0
3,0,246,139,11449,0
4,1,145,139,36,0


Отлично мы сделали предсказания пола, теперь посчитаем,насколько сильно мы ошиблись.

In [124]:
df['функция_потерь_для_пола'] = df[['пол', 'предсказанный_пол']].apply(lambda x: loss_classify(x[1], x[0]), axis=1)

In [134]:
df.head()

,пол,рост,предсказанный_рост,функция_потерь_для_роста,предсказанный_пол,функция_потерь_для_пола
0,0,139,139,0,0,False
1,1,223,139,7056,0,True
2,1,128,139,121,0,True
3,0,246,139,11449,0,False
4,1,145,139,36,0,True


А теперь и эмперический риск посчитать можно.

In [126]:
empirical_risk(df['функция_потерь_для_пола'].values)

0.504

Давайте как в задаче с ростом попробуем улучшить предсказания.
Попробуем предсказывать самым частым полом в датасете.

In [141]:
#здесь должен быть код с расчетом самого частого значения пола
df['предсказанный_пол'] = df['пол'].value_counts().keys()[0]

In [142]:
#здесь должен быть код с расчетом функции потерь для каждого ряда датасета
df['функция_потерь_для_пола'] = df[['пол', 'предсказанный_пол']].apply(lambda x: loss_classify(x[1], x[0]), axis=1)

In [143]:
#здесь должен быть код с расчетом функционала качества  (это и будет ответом)
empirical_risk(df['функция_потерь_для_пола'].values)

0.496

### Итог:

Мы решили целых две задачи предсказания. Задачу предсказания регрессии, а также задачу бинарной классификации.
Посчитали ошибки нашего алгоритма и оценили насколько мы ошибаемся на всей обучающей выборке.